# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
import os
import joblib

from azureml.train.automl import AutoMLConfig
from pprint import pprint # Used in printing automl model parameters
from azureml.core import Model # Used to get model information

## Dataset

### Overview

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

data_loc = "https://raw.githubusercontent.com/khalidw/Capstone-Project-Azure-Machine-Learning-Engineer/master/divorce.csv"
ds = TabularDatasetFactory.from_delimited_files(data_loc)
data = ds.to_pandas_dataframe().dropna()

In [4]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneExp'

experiment=Experiment(ws, experiment_name)

In [5]:
# Compute name should contain only letters, digits, hyphen and should be 2-16 charachters long
compute_name = "optCompute"
try:
    trainCluster = ComputeTarget(ws, compute_name)
    print(f"{compute_name} exists already")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=5)
    trainCluster = ComputeTarget.create(ws, compute_name, compute_config)
trainCluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

In [6]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
        task='classification',
        compute_target=trainCluster,
        training_data=ds,
        label_column_name='Class',
        n_cross_validations=5,
        **automl_settings
)

In [7]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneExp,AutoML_3c3d0d78-f1e8-490e-a8d9-3d58f1b64473,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
CapstoneExp,AutoML_3c3d0d78-f1e8-490e-a8d9-3d58f1b64473,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_3c3d0d78-f1e8-490e-a8d9-3d58f1b64473',
 'target': 'optCompute',
 'status': 'Completed',
 'startTimeUtc': '2023-01-04T13:37:38.572288Z',
 'endTimeUtc': '2023-01-04T13:56:39.472481Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'optCompute',
  'AMLSettingsJsonString': '{"path":null,"name":"CapstoneExp","subscription_id":"976ee174-3882-4721-b90a-b5fef6b72f24","resource_group":"aml-quickstarts-221321","workspace_name":"quick-starts-ws-221321","region":"southcentralus","compute_target":"optCompute","spark_s

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
# Get best run and model
best_run, fitted_model = remote_run.get_output()

# Print the best run
print(best_run)

# Get all metrics of the best run
best_run_metrics = best_run.get_metrics()

# Print all metrics of the best run
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Package:azureml-automl-runtime, training version:1.48.0.post1, current version:1.47.0
Package:azureml-core, training version:1.48.0, current version:1.47.0
Package:azureml-dataprep, training version:4.8.3, current version:4.5.7
Package:azureml-dataprep-rslex, training version:2.15.1, current version:2.11.4
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.47.0
Package:azureml-defaults, training version:1.48.0, current version:1.47.0
Package:azureml-interpret, training version:1.48.0, current version:1.47.0
Package:azureml-mlflow, training version:1.48.0, current version:1.47.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.47.0
Package:azureml-responsibleai, training version:1.48.0, current version:1.47.0
Package:azureml-telemetry, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-runtime, training version:1.48.0, current v

Run(Experiment: CapstoneExp,
Id: AutoML_3c3d0d78-f1e8-490e-a8d9-3d58f1b64473_24,
Type: azureml.scriptrun,
Status: Completed)
norm_macro_recall 0.9866666666666667
precision_score_weighted 0.9944117647058823
average_precision_score_weighted 1.0
f1_score_micro 0.9941176470588236
recall_score_micro 0.9941176470588236
precision_score_micro 0.9941176470588236
log_loss 0.11853129536805959
average_precision_score_micro 0.9998319327731092
recall_score_macro 0.9933333333333334
f1_score_weighted 0.9940916419618245
AUC_micro 0.9998269896193772
weighted_accuracy 0.9948805460750852
accuracy 0.9941176470588236
average_precision_score_macro 1.0
precision_score_macro 0.9949999999999999
f1_score_macro 0.9939876215738284
balanced_accuracy 0.9933333333333334
AUC_weighted 1.0
recall_score_weighted 0.9941176470588236
AUC_macro 1.0
matthews_correlation 0.9883259585576738
confusion_matrix aml://artifactId/ExperimentRun/dcid.AutoML_3c3d0d78-f1e8-490e-a8d9-3d58f1b64473_24/confusion_matrix
accuracy_table aml://a

In [11]:
# Print detailed parameters of the fitted model
def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook221321/code/Users/odl_user_221321/Capstone-Project-Azml'}

TruncatedSVDWrapper
{'n_components': 0.05947368421052632, 'random_state': None}

XGBoostClassifier
{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 0.7,
 'eta': 0.1,
 'gamma': 0,
 'gpu_id': -1,
 'grow_policy': 'lossguide',
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.100000001,
 'max_bin': 255,
 'max_delta_step': 0,
 'max_depth': 2,
 'max_leaves': 3,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 25,
 'n_jobs': 1,
 'num_p

In [12]:
#TODO: Save the best model
myModel = best_run.register_model(model_path='outputs/model.pkl', model_name='capstoneModel_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'Accuracy': best_run_metrics['accuracy']})

print(myModel)

Model(workspace=Workspace.create(name='quick-starts-ws-221321', subscription_id='976ee174-3882-4721-b90a-b5fef6b72f24', resource_group='aml-quickstarts-221321'), name=capstoneModel_automl, id=capstoneModel_automl:1, version=1, tags={'Training context': 'Auto ML'}, properties={'Accuracy': '0.9941176470588236'})


In [13]:
# List registered models to verify if model has been saved
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

capstoneModel_automl version: 1
	 Training context : Auto ML
	 Accuracy : 0.9941176470588236




## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'scoreScript.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'envFile.yml')

In [15]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script='scoreScript.py',
                                    environment=best_run.get_environment())

# deploy
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
service = Model.deploy(ws, "myservice", [myModel], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

print(service.scoring_uri)

print(service.swagger_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-01-04 14:18:16+00:00 Creating Container Registry if not exists..
2023-01-04 14:28:17+00:00 Registering the environment..
2023-01-04 14:28:18+00:00 Use the existing image.
2023-01-04 14:28:19+00:00 Submitting deployment to compute.
2023-01-04 14:28:22+00:00 Checking the status of deployment myservice..
2023-01-04 14:32:59+00:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
http://497e95c1-eb61-4291-bd39-8e9d91147d8f.southcentralus.azurecontainer.io/score
http://497e95c1-eb61-4291-bd39-8e9d91147d8f.southcentralus.azurecontainer.io/swagger.json


In [16]:
import json

#Import test data
test_df = data.sample(5) # data is the pandas dataframe of the original data
label_df = test_df.pop('Class')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"Atr1": 0, "Atr2": 0, "Atr3": 1, "Atr4": 0, "Atr5": 0, "Atr6": 2, "Atr7": 0, "Atr8": 0, "Atr9": 0, "Atr10": 1, "Atr11": 0, "Atr12": 2, "Atr13": 1, "Atr14": 0, "Atr15": 2, "Atr16": 0, "Atr17": 2, "Atr18": 1, "Atr19": 0, "Atr20": 1, "Atr21": 0, "Atr22": 0, "Atr23": 0, "Atr24": 0, "Atr25": 2, "Atr26": 2, "Atr27": 0, "Atr28": 0, "Atr29": 0, "Atr30": 0, "Atr31": 4, "Atr32": 1, "Atr33": 1, "Atr34": 1, "Atr35": 1, "Atr36": 1, "Atr37": 1, "Atr38": 2, "Atr39": 0, "Atr40": 2, "Atr41": 2, "Atr42": 1, "Atr43": 2, "Atr44": 3, "Atr45": 0, "Atr46": 2, "Atr47": 2, "Atr48": 1, "Atr49": 2, "Atr50": 1, "Atr51": 1, "Atr52": 1, "Atr53": 2, "Atr54": 0}, {"Atr1": 0, "Atr2": 0, "Atr3": 0, "Atr4": 0, "Atr5": 0, "Atr6": 0, "Atr7": 0, "Atr8": 0, "Atr9": 0, "Atr10": 0, "Atr11": 0, "Atr12": 0, "Atr13": 0, "Atr14": 0, "Atr15": 0, "Atr16": 0, "Atr17": 0, "Atr18": 0, "Atr19": 0, "Atr20": 0, "Atr21": 0, "Atr22": 0, "Atr23": 0, "Atr24": 0, "Atr25": 0, "Atr26": 0, "Atr27": 0, "Atr28": 0, "Atr29": 0, "Atr30": 

TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
import requests # Used for http post request

# Set the content type
headers = {'Content-type': 'application/json'}


response = requests.post(service.scoring_uri, test_sample, headers=headers)

In [18]:
# Print results from the inference
print(response.text)

"{\"result\": [1, 0, 1, 0, 1]}"


In [19]:
# Print original labels
print(label_df)

5      1
102    0
61     1
135    0
30     1
Name: Class, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [20]:
print(service.get_logs())

2023-01-04T14:32:31,230494181+00:00 - rsyslog/run 
2023-01-04T14:32:31,230469378+00:00 - gunicorn/run 
2023-01-04T14:32:31,230458276+00:00 - iot-server/run 
2023-01-04T14:32:31,233392600+00:00 | gunicorn/run | 
2023-01-04T14:32:31,235592617+00:00 | gunicorn/run | ###############################################
2023-01-04T14:32:31,241544176+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-01-04T14:32:31,246621609+00:00 | gunicorn/run | ###############################################
2023-01-04T14:32:31,253401188+00:00 | gunicorn/run | 
2023-01-04T14:32:31,257930341+00:00 | gunicorn/run | 
2023-01-04T14:32:31,282544994+00:00 | gunicorn/run | AzureML image information: openmpi3.1.2-ubuntu18.04, Materializaton Build:20221129.v2
2023-01-04T14:32:31,286570775+00:00 | gunicorn/run | 
2023-01-04T14:32:31,291393371+00:00 | gunicorn/run | 
2023-01-04T14:32:31,301847280+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml_4ceb345e4e0a419c28b56298b234f6b5/bin:/o

In [21]:
service.delete()